In [ ]:
!pip3 install opencv-python 

# Imports

In [4]:
import matplotlib.pyplot as plt
import random
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils, optimizers
from PIL import Image
import shutil
from tensorflow.keras import models, layers
from tensorflow.keras.applications import vgg16
from tensorflow.keras.callbacks import EarlyStopping

# Create DataFrame

In [5]:
#These variables can be changes, excluding train_dir

train_dir = "../raw_data/Training"
img_height, img_width = 256, 256
batch_size = 32

In [6]:
#Splits into train_generator and validation_generator
#This bulk uploads the images
#Creates target (y) for us!

#Play around with the interpolation argument - bicubic, lanczos??? 

train_datagen = ImageDataGenerator(rescale=1./255,
                                    vertical_flip=True,
                                    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
                                    train_dir,
                                    target_size=(img_height, img_width),
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    subset='training',
                                    keep_aspect_ratio=True,
                                    interpolation='lanczos') # set as training data

validation_generator = train_datagen.flow_from_directory(
                                    train_dir, # same directory as training data
                                    target_size=(img_height, img_width),
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    subset='validation',
                                    keep_aspect_ratio=True,
                                    interpolation='lanczos') # set as validation data

Found 9677 images belonging to 83 classes.
Found 2380 images belonging to 83 classes.


# Model

In [7]:
...
# Model needs building + transfer learning  
...
def initialize_model():
    base_model = vgg16.VGG16(
                        include_top=False,
                        weights='imagenet',
                        input_shape=(img_height, img_width, 3),
                        classifier_activation='softmax')
    
    base_model.trainable = False
    
    model = models.Sequential([ 
        base_model,
        layers.Flatten(),
        layers.Dense(500, activation="relu"),
        layers.Dense(83, activation="softmax")
    ])
    
    
    opt = optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,
                loss='categorical_crossentropy',
                metrics=['accuracy']) 
    
    return model

In [8]:
model = initialize_model()

2023-03-09 09:15:00.395810: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/dolly/.pyenv/versions/3.10.6/envs/YOGi/lib/python3.10/site-packages/cv2/../../lib64:
2023-03-09 09:15:00.395868: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-09 09:15:00.395885: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Dolly): /proc/driver/nvidia/version does not exist
2023-03-09 09:15:00.396181: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebui

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 500)               16384500  
                                                                 
 dense_1 (Dense)             (None, 83)                41583     
                                                                 
Total params: 31,140,771
Trainable params: 16,426,083
Non-trainable params: 14,714,688
_________________________________________________________________


In [9]:
es = EarlyStopping(patience=1)

In [10]:
#fit model - fit on train_generator (both X and y) and the validation data is validation_generator
history = model.fit(
                train_generator,
                steps_per_epoch = train_generator.samples // batch_size,
                validation_data = validation_generator, 
                validation_steps = validation_generator.samples // batch_size,
                epochs = 2,
                callbacks=[es])

Epoch 1/2


/home/dolly/.pyenv/versions/3.10.6/envs/YOGi/lib/python3.10/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


  1/302 [..............................] - ETA: 31:23 - loss: 4.8094 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label='train' + exp_name)
    ax1.plot(history.history['val_loss'], label='val' + exp_name)
    #ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    #ax2.set_ylim(0.25, 1.)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

In [ ]:
plot_history(history)

In [ ]:
test_dir = f"{root_dir}/Testing"

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255) # set validation split

test_generator = test_datagen.flow_from_directory(
                                    test_dir,
                                    target_size=(img_height, img_width),
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    subset='training',
                                    keep_aspect_ratio=True,
                                    interpolation='lanczos') # set as training data

In [ ]:
model.evaluate(test_generator)

# Bounding Boxes

In [ ]:
#This has been paused and probably won't be used

In [ ]:
im = cv2.imread(f"{file_path}/{file_list[0]}")

In [ ]:
# Reading the Image
image = cv2.imread(f"{file_path}/{file_list[3]}")

# initialize the HOG descriptor
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

(humans, _) = hog.detectMultiScale(image, winStride=(3,3),
                                padding=(32, 32), 
                               scale=1.01)

# getting no. of human detected
print('Human Detected : ', len(humans))

# loop over all detected humans
for (x, y, w, h) in humans:
   pad_w, pad_h = int(0.15 * w), int(0.01 * h)
   cv2.rectangle(image, (x + pad_w, y + pad_h), (x + w - pad_w, y + h - pad_h), (0, 255, 0), 2)



In [ ]:
# display the output image
cv2.imshow("Image", image)
cv2.waitKey(25)
# cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
filepath = f"{file_path}/{file_list[3]}"
photo = cv2.imread(filepath)
photo_grey = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)

In [ ]:
# Initializing the HOG person
# detector

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
   
# Reading the Image
image = cv2.imread(filepath)
   
# Detecting all the regions in the 
# Image that has a pedestrians inside it
(regions, _) = hog.detectMultiScale(image, 
                                    winStride=(8, 8),
                                    padding=(16, 16),
                                    scale=1.15)
   
# Drawing the regions in the Image
for (x, y, w, h) in regions:
    cv2.rectangle(image, (x, y), 
                  (x + w, y + h), 
                  (0, 0, 255), 2)
    
#Showing the output Image
cv2.imshow("Image", image)
cv2.waitKey(0)
   
cv2.destroyAllWindows()